<a href="https://www.kaggle.com/code/ayushkhaire/crypto-dataset-daily-updates-top-1000?scriptVersionId=196044826" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import requests as rq
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
from tqdm import tqdm
from kaggle_secrets import UserSecretsClient
from datetime import datetime
import json as js
import subprocess
import logging
import time
import shutil

# Yeasterday data :)

In [2]:
df = pd.read_csv('/kaggle/input/top-1000-cryptos-historical/1000_cryptos.csv')
df

,dates,symbol,open,close,high,low,volume,adj_close
0,2015-01-01 00:00:00,BTC-USD,320.434998,314.248993,320.434998,314.002991,8036550.0,314.248993
1,2015-01-02 00:00:00,BTC-USD,314.079010,315.032013,315.838989,313.565002,7860650.0,315.032013
2,2015-01-03 00:00:00,BTC-USD,314.846008,281.082001,315.149994,281.082001,33054400.0,281.082001
3,2015-01-04 00:00:00,BTC-USD,281.145996,264.195007,287.230011,257.612000,55629100.0,264.195007
4,2015-01-05 00:00:00,BTC-USD,265.084015,274.473999,278.341003,265.084015,43962800.0,274.473999
...,...,...,...,...,...,...,...,...
2138644,2024-09-05 00:00:00,ETHO-USD,0.007804,0.006347,0.008318,0.006042,70268.0,0.006347
2138645,2024-09-06 00:00:00,ETHO-USD,0.006347,0.006860,0.007803,0.006045,70166.0,0.006860
2138646,2024-09-07 00:00:00,ETHO-USD,0.006860,0.007045,0.007986,0.005761,76601.0,0.007045
2138647,2024-09-08 00:00:00,ETHO-USD,0.007045,0.006926,0.007992,0.005733,79452.0,0.006926


# Define headers

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Extract symbols

In [4]:
 link = f'https://finance.yahoo.com/markets/crypto/all/?start=0&count=100'

# Get symbols and hit urls

In [5]:
resp_main_list = []
i = 0
for page in tqdm(range(20)):
    url = f'https://finance.yahoo.com/markets/crypto/all/?start={i*100}&count=100'
    r = rq.get(url,headers = headers)
    resp_main_list.append(r)
    i = i+1
#     print(url)

100%|██████████| 20/20 [00:08<00:00,  2.46it/s]


In [6]:
symbol_list = []

for r in tqdm(resp_main_list):
    soup = BeautifulSoup(r.text,'html.parser')
    symbs= soup.find_all('span',{'class':'symbol yf-ravs5v'})
    for s in symbs:
        symbol_list.append(s.text)
        
print(len(symbol_list))

100%|██████████| 20/20 [00:13<00:00,  1.54it/s]

2000


# Now , get the link for today downloads from a tesla stock 

### This we willl use to use "code" by yahoo finance to figure out dates 

In [7]:
'''

llink1 = https://query1.finance.yahoo.com/v8/finance/chart/GLM-USD?events=capitalGain%7Cdiv%7Csplit&formatted=true&includeAdjustedClose=true&interval=1d&period1=1510185600&period2=1725897789&symbol=GLM-USD&userYfid=true&lang=en-US&region=US

here the period values are exactly same.
'''

# After research , i got they are unix timestamps in period . lets try it out !

import time

# Get current Unix timestamp
current_timestamp = int(time.time())

date_str = "2015-01-01"
date_obj = datetime.strptime(date_str, "%Y-%m-%d")
period1 = int(time.mktime(date_obj.timetuple()))

period2 = current_timestamp  

# Print the timestamps in the desired format
print(f"period1={period1}&period2={period2}")


period1=1420070400&period2=1725960242


# create stocks directory

In [8]:
os.mkdir('crypto_data')

# Download all files

In [9]:
def return_timestamp(unix_timestamps, date_format='%Y-%m-%d %H:%M:%S'):
    formatted_dates = []
    for ut in unix_timestamps:
        date = datetime.fromtimestamp(ut)
        formatted_date = date.strftime(date_format)
        formatted_dates.append(formatted_date)
    return formatted_dates

print(return_timestamp([1725898133]))

['2024-09-09 16:08:53']


In [10]:
for stock in tqdm(symbol_list):
    url = f'https://query1.finance.yahoo.com/v8/finance/chart/{stock}?events=capitalGain%7Cdiv%7Csplit&formatted=true&includeAdjustedClose=true&interval=1d&period1={period1}&period2={period2}&symbol={stock}&userYfid=true&lang=en-US&region=US'
#     print(url)
    r = rq.get(url, headers=headers)
    if r.status_code == 200:
        with open(f"crypto_data/{stock}.json", "wb") as file:
            file.write(r.content)
#         print(f"File downloaded successfully : {stock}.json")
    else:
#         print(f"Failed to download file : {stock}.json")
        pass
#     time.sleep(3)

100%|██████████| 2000/2000 [03:02<00:00, 10.94it/s]


In [11]:
symbol_list_dir = os.listdir('/kaggle/working/crypto_data/')
len(symbol_list_dir)

2000

# Read a sample

In [12]:
jsonf = pd.read_json('/kaggle/working/crypto_data/BTC-USD.json')
jsondict = jsonf.to_dict()
timestamp = jsondict.get('chart').get('result')[0].get('timestamp')
sym = jsondict.get('chart').get('result')[0].get('meta').get('symbol')
symbol = list(sym for s in range(len(timestamp)))
low = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('low')
high = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('high')
volume = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('volume')
open_ = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('open')
close = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('close')
adj_close = jsondict.get('chart').get('result')[0].get('indicators').get('adjclose')[0].get('adjclose')
dates = return_timestamp(timestamp)
tempdf = pd.DataFrame(
    {
    'dates':dates,
    'symbol':symbol,
    'open':open_,
    'close':close,
    'high':high,
    'low':low,
    'volume':volume,
    'adj_close':adj_close
    }
)
tempdf

,dates,symbol,open,close,high,low,volume,adj_close
0,2015-01-01 00:00:00,BTC-USD,320.434998,314.248993,320.434998,314.002991,8036550,314.248993
1,2015-01-02 00:00:00,BTC-USD,314.079010,315.032013,315.838989,313.565002,7860650,315.032013
2,2015-01-03 00:00:00,BTC-USD,314.846008,281.082001,315.149994,281.082001,33054400,281.082001
3,2015-01-04 00:00:00,BTC-USD,281.145996,264.195007,287.230011,257.612000,55629100,264.195007
4,2015-01-05 00:00:00,BTC-USD,265.084015,274.473999,278.341003,265.084015,43962800,274.473999
...,...,...,...,...,...,...,...,...
3536,2024-09-06 00:00:00,BTC-USD,56160.191406,53948.753906,56976.109375,52598.699219,49361693566,53948.753906
3537,2024-09-07 00:00:00,BTC-USD,53949.085938,54139.687500,54838.144531,53740.070312,19061486526,54139.687500
3538,2024-09-08 00:00:00,BTC-USD,54147.933594,54841.566406,55300.859375,53653.757812,18268287531,54841.566406
3539,2024-09-09 00:00:00,BTC-USD,54851.886719,57019.535156,58041.125000,54598.433594,34618096173,57019.535156


# Work on dataset

In [13]:
megadf = pd.DataFrame()

In [14]:
for jso in tqdm(symbol_list_dir):
    jsonf = pd.read_json(f'/kaggle/working/crypto_data/{jso}')
    jsondict = jsonf.to_dict()
    
    # Safely get the timestamp and check if it's None
    timestamp = jsondict.get('chart').get('result')[0].get('timestamp')
    if timestamp is None:
        print(f"Missing timestamp for {jso}. Skipping this symbol.")
        continue  # Skip this iteration if timestamp is None
    
    # Proceed with the rest of the code after ensuring timestamp is not None
    low = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('low')
    sym = jsondict.get('chart').get('result')[0].get('meta').get('symbol')
    
    # Create a list of symbols based on the length of timestamp
    symbol = list(sym for s in range(len(timestamp)))
    
    high = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('high')
    volume = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('volume')
    open_ = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('open')
    close = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('close')
    adj_close = jsondict.get('chart').get('result')[0].get('indicators').get('adjclose')[0].get('adjclose')
    
    # Convert timestamp to dates
    dates = return_timestamp(timestamp)
    
    # Create a DataFrame
    tempdf = pd.DataFrame(
    {
        'dates': dates,
        'symbol': symbol,
        'open': open_,
        'close': close,
        'high': high,
        'low': low,
        'volume': volume,
        'adj_close': adj_close
    })
    
    # Concatenate the new data to the main DataFrame
    megadf = pd.concat([megadf, tempdf], ignore_index=True)


 17%|█▋        | 342/2000 [00:08<00:52, 31.44it/s]

Missing timestamp for TADDY-USD.json. Skipping this symbol.


 34%|███▍      | 676/2000 [00:22<00:55, 23.96it/s]

Missing timestamp for SUNPEPE-USD.json. Skipping this symbol.


 41%|████      | 823/2000 [00:30<00:53, 21.99it/s]

...

 96%|█████████▋| 1930/2000 [02:16<00:09,  7.22it/s]

Missing timestamp for MOBY32259-USD.json. Skipping this symbol.


 99%|█████████▉| 1981/2000 [02:22<00:02,  9.08it/s]

Missing timestamp for SOY32350-USD.json. Skipping this symbol.


100%|██████████| 2000/2000 [02:25<00:00, 13.75it/s]


In [15]:
megadf['symbol'].unique()

array(['DF-USD', 'GNO-USD', 'MIR7857-USD', ..., 'BRWL-USD', 'RACA-USD',
       'HSF-USD'], dtype=object)

In [16]:
first_date = megadf['dates'][0]
last_date = megadf['dates'].iloc[-1]

In [17]:
first_date,last_date

('2020-06-22 00:00:00', '2024-09-10 09:22:00')

In [18]:
megadf.dtypes

dates         object
symbol        object
open         float64
close        float64
high         float64
low          float64
volume       float64
adj_close    float64
dtype: object

In [19]:
megadf['dates'] = pd.to_datetime(megadf['dates'], format="mixed")

In [20]:
megadf.dtypes

dates        datetime64[ns]
symbol               object
open                float64
close               float64
high                float64
low                 float64
volume              float64
adj_close           float64
dtype: object

In [21]:
megadf

,dates,symbol,open,close,high,low,volume,adj_close
0,2020-06-22 00:00:00,DF-USD,0.372348,0.360021,0.376862,0.297726,240181.0,0.360021
1,2020-06-23 00:00:00,DF-USD,0.361614,0.393201,0.445981,0.321671,149736.0,0.393201
2,2020-06-24 00:00:00,DF-USD,0.388931,0.483492,0.630291,0.383294,516735.0,0.483492
3,2020-06-25 00:00:00,DF-USD,0.470206,0.426568,0.497649,0.405323,227588.0,0.426568
4,2020-06-26 00:00:00,DF-USD,0.426314,0.422367,0.489849,0.385589,225487.0,0.422367
...,...,...,...,...,...,...,...,...
2144582,2024-09-06 00:00:00,HSF-USD,0.072774,0.063910,0.106295,0.062871,684075.0,0.063910
2144583,2024-09-07 00:00:00,HSF-USD,0.063910,0.066773,0.086479,0.062610,671960.0,0.066773
2144584,2024-09-08 00:00:00,HSF-USD,0.066773,0.077838,0.091729,0.066773,667987.0,0.077838
2144585,2024-09-09 00:00:00,HSF-USD,0.077838,0.108826,0.136197,0.077828,697006.0,0.108826


In [22]:
os.mkdir('dataset')

In [23]:
megadf.to_csv(f'dataset/1000_cryptos.csv',index = False)

# final dataset

In [24]:
df = pd.read_csv(f'/kaggle/working/dataset/1000_cryptos.csv')

In [25]:
df

,dates,symbol,open,close,high,low,volume,adj_close
0,2020-06-22 00:00:00,DF-USD,0.372348,0.360021,0.376862,0.297726,240181.0,0.360021
1,2020-06-23 00:00:00,DF-USD,0.361614,0.393201,0.445981,0.321671,149736.0,0.393201
2,2020-06-24 00:00:00,DF-USD,0.388931,0.483492,0.630291,0.383294,516735.0,0.483492
3,2020-06-25 00:00:00,DF-USD,0.470206,0.426568,0.497649,0.405323,227588.0,0.426568
4,2020-06-26 00:00:00,DF-USD,0.426314,0.422367,0.489849,0.385589,225487.0,0.422367
...,...,...,...,...,...,...,...,...
2144582,2024-09-06 00:00:00,HSF-USD,0.072774,0.063910,0.106295,0.062871,684075.0,0.063910
2144583,2024-09-07 00:00:00,HSF-USD,0.063910,0.066773,0.086479,0.062610,671960.0,0.066773
2144584,2024-09-08 00:00:00,HSF-USD,0.066773,0.077838,0.091729,0.066773,667987.0,0.077838
2144585,2024-09-09 00:00:00,HSF-USD,0.077838,0.108826,0.136197,0.077828,697006.0,0.108826


# Upload Dataset

# User secrets

In [26]:
user_secrets = UserSecretsClient()
kaggle_apikey = user_secrets.get_secret("kaggle_apikey")
kaggle_username = user_secrets.get_secret("kaggle_username")

os.environ['KAGGLE_USERNAME'] = kaggle_username
os.environ['KAGGLE_KEY'] = kaggle_apikey

# Metadata file

In [27]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print('Starting upload >>>>>>') 
print('Creating metadata file >>>>')
data = {
    "id": "ayushkhaire/top-1000-cryptos-historical"
}
metadata_file_location = '/kaggle/working/dataset/dataset-metadata.json' 
with open(metadata_file_location, 'w', encoding='utf-8') as metadata_file:
    js.dump(data, metadata_file)
print('Metadata file created')

Starting upload >>>>>>
Creating metadata file >>>>
Metadata file created


# upload

In [28]:
retries = 0
while retries < 5:
    try:
        command = "kaggle datasets version -p '/kaggle/working/dataset' -m 'Update' -r zip"
        subprocess.run(command, shell=True, check=True)
        logger.info("Upload complete")
        break
    except Exception as error:
        logger.error(f"Error from Kaggle: {error}")
        time.sleep(5)
        retries += 1

Starting upload for file 1000_cryptos.csv


100%|██████████| 268M/268M [00:04<00:00, 65.0MB/s]


Upload successful: 1000_cryptos.csv (268MB)
Dataset version is being created. Please check progress at https://www.kaggle.com/ayushkhaire/top-1000-cryptos-historical


In [29]:
shutil.rmtree('/kaggle/working/crypto_data')
shutil.rmtree('/kaggle/working/dataset')

# Thank you , and I will see you in next notebook :)